# Infinite Loop

Your flight to the major airline hub reaches cruising altitude without incident. While you consider checking the in-flight menu for one of those drinks that come with a little umbrella, you are interrupted by the kid sitting next to you.

Their handheld game console won't turn on! They ask if you can take a look.

You narrow the problem down to a strange infinite loop in the boot code (your puzzle input) of the device. You should be able to fix it, but first you need to be able to run the code in isolation.

The boot code is represented as a text file with one instruction per line of text. Each instruction consists of an operation (acc, jmp, or nop) and an argument (a signed number like +4 or -20).

acc increases or decreases a single global value called the accumulator by the value given in the argument. For example, acc +7 would increase the accumulator by 7. The accumulator starts at 0. After an acc instruction, the instruction immediately below it is executed next.
jmp jumps to a new instruction relative to itself. The next instruction to execute is found using the argument as an offset from the jmp instruction; for example, jmp +2 would skip the next instruction, jmp +1 would continue to the instruction immediately below it, and jmp -20 would cause the instruction 20 lines above to be executed next.
nop stands for No OPeration - it does nothing. The instruction immediately below it is executed next.
For example, consider the following program:

nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
These instructions are visited in this order:

nop +0  | 1
acc +1  | 2, 8(!)
jmp +4  | 3
acc +3  | 6
jmp -3  | 7
acc -99 |
acc +1  | 4
jmp -4  | 5
acc +6  |
First, the nop +0 does nothing. Then, the accumulator is increased from 0 to 1 (acc +1) and jmp +4 sets the next instruction to the other acc +1 near the bottom. After it increases the accumulator from 1 to 2, jmp -4 executes, setting the next instruction to the only acc +3. It sets the accumulator to 5, and jmp -3 causes the program to continue back at the first acc +1.

This is an infinite loop: with this sequence of jumps, the program will run forever. The moment the program tries to run any instruction a second time, you know it will never terminate.

Immediately before the program would run an instruction a second time, the value in the accumulator is 5.

Run your copy of the boot code. Immediately before any instruction is executed a second time, what value is in the accumulator?

## Second part

After some careful analysis, you believe that exactly one instruction is corrupted.

Somewhere in the program, either a jmp is supposed to be a nop, or a nop is supposed to be a jmp. (No acc instructions were harmed in the corruption of this boot code.)

The program is supposed to terminate by attempting to execute an instruction immediately after the last instruction in the file. By changing exactly one jmp or nop, you can repair the boot code and make it terminate correctly.

For example, consider the same program from above:

nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6
If you change the first instruction from nop +0 to jmp +0, it would create a single-instruction infinite loop, never leaving that instruction. If you change almost any of the jmp instructions, the program will still eventually find another jmp instruction and loop forever.

However, if you change the second-to-last instruction (from jmp -4 to nop -4), the program terminates! The instructions are visited in this order:

nop +0  | 1
acc +1  | 2
jmp +4  | 3
acc +3  |
jmp -3  |
acc -99 |
acc +1  | 4
nop -4  | 5
acc +6  | 6
After the last instruction (acc +6), the program terminates by attempting to run the instruction below the last instruction in the file. With this change, after the program terminates, the accumulator contains the value 8 (acc +1, acc +1, acc +6).

Fix the program so that it terminates normally by changing exactly one jmp (to nop) or nop (to jmp). What is the value of the accumulator after the program terminates?

In [1]:
import os
import time
import numpy as np
import re

In [2]:
DAY = 'Day_08'
FILE_END = '_01.txt'
RUN_WITH = 'sample' ## admitted values: sample input 
SAMPLE_DATA = '''nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6'''

In [3]:
start = time.time()

In [4]:

input_path = os.path.join(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))), "Inputs")
input_path_day = os.path.join(input_path, DAY)
file_path = os.path.join(input_path_day, DAY+ FILE_END)

### Read input

In [5]:
if RUN_WITH == 'input':
    with open(file_path,'r') as f:
        values = f.read()
elif RUN_WITH == 'sample':
    values =  SAMPLE_DATA
    
else:
    print("No valid input selected")
values = values.split('\n')
try:
    values.remove("")
except:
    pass
values[0:5]
mid_time = time.time()

## Get Instructions:

In [6]:
instructions = {i:(k.split()[0], int(k.split()[1])) for i,k in enumerate(values)}

# Part I:

In [7]:
def solve_part_1(instructions):
    instructions = instructions.copy()
    acc_val = 0
    positions_visited = set()
    next_position = 0
    curr_position = 0
    while True:
        curr_position = next_position
        positions_visited.add(curr_position)
        curr_instruction = instructions[curr_position][0]
        curr_val = instructions[curr_position][1]
        if curr_instruction in ('nop', 'acc'):
            next_position += 1
        elif curr_instruction in ('jmp'):
            next_position += curr_val
        if curr_instruction =='acc':
            acc_val += curr_val
        if next_position in positions_visited:
            break
    print(f'Acc_val = {acc_val},\tLast position = {curr_position}' )
    return positions_visited, curr_position, next_position, acc_val

In [8]:
# 1723

In [9]:
positions_visited, curr_position, next_position,  acc_val = solve_part_1(instructions)
acc_val, curr_position, next_position

Acc_val = 5,	Last position = 4


(5, 4, 1)

## Part II:

In [10]:
# Part II requires that Part I has been ran first

In [11]:
# Positions visited that were jmp/nop
to_test = [x for x in positions_visited if instructions[x][0] in ('jmp','nop')]

In [12]:
def solve_part_2(instructions):
    # Modified version of the solve_part_1
    acc_val = 0
    positions_visited = set()
    next_position = 0
    curr_position = 0
    completed = False
    while True:
        curr_position = next_position
        positions_visited.add(next_position)
        curr_instruction = instructions[curr_position][0]
        curr_val = instructions[curr_position][1]
        if curr_instruction in ('nop', 'acc'):
            next_position += 1
        elif curr_instruction in ('jmp'):
            next_position += curr_val
        if curr_instruction =='acc':
            acc_val += curr_val
        if next_position in positions_visited:
            break
        if next_position >= len(instructions):
            completed = True
            break
    #print(f'Acc_val = {acc_val},\tLast position = {curr_position}' )
    return positions_visited, curr_position, next_position, completed, acc_val

In [13]:
completed = False
while not completed and to_test:
    # try all the positions visited that were jmp or nop until the instructions are completed
    change = to_test.pop()
    new_instructions = instructions.copy()
    value = new_instructions[change] 
    if value[0] == 'jmp':
        new_instructions[change] = ('nop', value[1])
    elif value[1] == 'nop':
        new_instructions[change] = ('jmp', value[1])
    tmp_positions_visited, tmp_curr_position, tmp_next_position, tmp_completed, tmp_acc_val = solve_part_2(new_instructions)
    if tmp_completed:
        print('completed')
        break
print(f'Acc_val = {tmp_acc_val}, Last_position = {tmp_curr_position}')

completed
Acc_val = 8, Last_position = 8


In [14]:
end = time.time()
read = mid_time - start
end - start

0.17300200462341309